In [1]:
import duckdb
import pandas as pd

In [2]:
%load_ext sql
conn = duckdb.connect()
%sql conn --alias duckdb

In [3]:
%sql CREATE TABLE houses AS SELECT * FROM read_json_auto('../dataset/houses-9k.json')

Running query in 'duckdb'

Count
9047


In [4]:
%sql SELECT * FROM houses LIMIT 1

Running query in 'duckdb'

id,price,installment,address,tags,description,specs,facilities,agent,images,url
hos13819879,295000.0,21.0,"Sentul City, Bogor","['Cash Keras', 'One Gate System']","Rumah siap masuk dengan style englang house,berada di dalam cluster di sentul city,lokasi sangat strategis dan cocok sekali untuk tinggal karena dekat dengan semua fasilitas di sentul city seperti;sekolah,tempat ibadah,mall,pasar tradisional dan pasar bersih dan hanya beberapa menit ke gerbang toll sentul city.","{'Kamar Tidur': 3, 'Kamar Mandi': 4, 'Luas Tanah': '400 m²', 'Luas Bangunan': '150 m²', 'Carport': 2, 'Tipe Properti': 'Rumah', 'Sertifikat': 'SHM - Sertifikat Hak Milik', 'Daya Listrik': '2200 Watt', 'Kamar Pembantu': 1, 'Kamar Mandi Pembantu': 1, 'Dapur': 1, 'Ruang Makan': 'Tidak', 'Ruang Tamu': 'Tidak', 'Kondisi Perabotan': 'Unfurnished', 'Material Bangunan': 'Bata Merah', 'Material Lantai': 'Granit', 'Garasi': 2, 'Jumlah Lantai': 2, 'Hadap': 'Timur', 'Konsep dan Gaya Rumah': 'Scandinavian', 'Pemandangan': 'Pemukiman Warga', 'Terjangkau Internet': 'Ya', 'Lebar Jalan': '2 Mobil', 'Tahun Dibangun': 2002, 'Sumber Air': 'PAM atau PDAM', 'Hook': 'Tidak', 'Kondisi Properti': 'Bagus', 'ID Iklan': 'hos13819879', 'Tahun Di Renovasi': None}","['Jalur Telepon', 'Taman', 'Tempat Jemuran', 'Akses Parkir', 'CCTV', 'Keamanan 24 jam', 'Kolam Renang', 'Masjid', 'One Gate System', 'Playground', 'Tempat Laundry', 'Kitchen Set', 'Lapangan Basket', 'Lapangan Bola', 'Lapangan Bulu Tangkis', 'Lapangan Tenis', 'Lapangan Voli', 'Tempat Gym', 'Track Lari']","{'name': 'Heri Andrian', 'url': 'https://www.rumah123.com/agen-properti/nas-global-property/heri-andrian-27716/', 'phone': '+628121859577', 'company': {'name': 'NAS Global Property', 'url': 'https://www.rumah123.com/agen-properti/nas-global-property-74334/'}}","['https://picture.rumah123.com/r123-images/720x420-crop/customer/27716/2023-05-25-03-11-04-e2b69471-1fde-4d0a-98d4-9b82fffbc3bb.jpg', 'https://picture.rumah123.com/r123-images/360x401-inside/customer/27716/2023-05-25-03-11-16-016f9d90-68f2-4bc0-9db9-8f4184978cbd.jpg', 'https://picture.rumah123.com/r123-images/360x401-inside/customer/27716/2023-05-25-03-11-17-3ef428f7-4ef1-4173-aee3-7e7ae23d4940.jpg', 'https://picture.rumah123.com/r123-images/360x401-inside/customer/27716/2023-05-25-03-11-19-4c2dba24-3136-460a-a57b-a1346ffb040b.jpg', 'https://picture.rumah123.com/r123-images/360x401-inside/customer/27716/2023-05-25-03-11-20-c9d0beca-a01c-4cc3-98cc-51c3ea4aec1d.jpg']",https://www.rumah123.com/properti/bogor/hos13819879/


In [5]:
%sql DESCRIBE SELECT * FROM houses

Running query in 'duckdb'

column_name,column_type,null,key,default,extra
id,VARCHAR,YES,None,None,None
price,DOUBLE,YES,None,None,None
installment,DOUBLE,YES,None,None,None
address,VARCHAR,YES,None,None,None
tags,VARCHAR[],YES,None,None,None
description,VARCHAR,YES,None,None,None
specs,"STRUCT(""Kamar Tidur"" BIGINT, ""Kamar Mandi"" BIGINT, ""Luas Tanah"" VARCHAR, ""Luas Bangunan"" VARCHAR, Carport BIGINT, ""Tipe Properti"" VARCHAR, Sertifikat VARCHAR, ""Daya Listrik"" VARCHAR, ""Kamar Pembantu"" BIGINT, ""Kamar Mandi Pembantu"" BIGINT, Dapur BIGINT, ""Ruang Makan"" VARCHAR, ""Ruang Tamu"" VARCHAR, ""Kondisi Perabotan"" VARCHAR, ""Material Bangunan"" VARCHAR, ""Material Lantai"" VARCHAR, Garasi BIGINT, ""Jumlah Lantai"" BIGINT, Hadap VARCHAR, ""Konsep dan Gaya Rumah"" VARCHAR, Pemandangan VARCHAR, ""Terjangkau Internet"" VARCHAR, ""Lebar Jalan"" VARCHAR, ""Tahun Dibangun"" BIGINT, ""Sumber Air"" VARCHAR, Hook VARCHAR, ""Kondisi Properti"" VARCHAR, ""ID Iklan"" VARCHAR, ""Tahun Di Renovasi"" BIGINT)",YES,None,None,None
facilities,VARCHAR[],YES,None,None,None
agent,"STRUCT(""name"" VARCHAR, url VARCHAR, phone VARCHAR, company STRUCT(""name"" VARCHAR, url VARCHAR))",YES,None,None,None
images,VARCHAR[],YES,None,None,None


In [13]:
conn.sql("SUMMARIZE SELECT specs.* FROM houses")

┌──────────────────────┬─────────────┬──────────────────────┬───┬─────────┬───────┬─────────────────┐
│     column_name      │ column_type │         min          │ … │   q75   │ count │ null_percentage │
│       varchar        │   varchar   │       varchar        │   │ varchar │ int64 │     varchar     │
├──────────────────────┼─────────────┼──────────────────────┼───┼─────────┼───────┼─────────────────┤
│ Kamar Tidur          │ BIGINT      │ 1                    │ … │ 4       │  9047 │ 2.56%           │
│ Kamar Mandi          │ BIGINT      │ 1                    │ … │ 3       │  9047 │ 2.39%           │
│ Luas Tanah           │ VARCHAR     │ 1 m²                 │ … │ NULL    │  9047 │ 0.01%           │
│ Luas Bangunan        │ VARCHAR     │ 1 m²                 │ … │ NULL    │  9047 │ 0.17%           │
│ Carport              │ BIGINT      │ 1                    │ … │ 2       │  9047 │ 35.27%          │
│ Tipe Properti        │ VARCHAR     │ Rumah                │ … │ NULL    │  9047 

In [7]:
%sql CREATE OR REPLACE VIEW houses_v1 AS SELECT id, COUNT(images), price, installment, split_part(address, ', ', 1) AS address_kecamatan, specs, facilities FROM houses GROUP BY id, price, installment, address_kecamatan, specs, facilities

Running query in 'duckdb'

Count


In [8]:
%sql SELECT * FROM houses_v1 LIMIT 2

Running query in 'duckdb'

id,count(images),price,installment,address_kecamatan,specs,facilities
hos15814995,1,970.0,6.0,Gunung Sindur,"{'Kamar Tidur': 2, 'Kamar Mandi': 2, 'Luas Tanah': '167 m²', 'Luas Bangunan': '165 m²', 'Carport': 1, 'Tipe Properti': 'Rumah', 'Sertifikat': 'SHM - Sertifikat Hak Milik', 'Daya Listrik': '3500 Watt', 'Kamar Pembantu': None, 'Kamar Mandi Pembantu': None, 'Dapur': 1, 'Ruang Makan': 'Ya', 'Ruang Tamu': 'Ya', 'Kondisi Perabotan': 'Unfurnished', 'Material Bangunan': None, 'Material Lantai': None, 'Garasi': None, 'Jumlah Lantai': 1, 'Hadap': 'Timur', 'Konsep dan Gaya Rumah': None, 'Pemandangan': None, 'Terjangkau Internet': 'Tidak', 'Lebar Jalan': None, 'Tahun Dibangun': None, 'Sumber Air': None, 'Hook': 'Tidak', 'Kondisi Properti': 'Bagus', 'ID Iklan': 'hos15814995', 'Tahun Di Renovasi': None}","['Taman', 'Tempat Jemuran', 'Keamanan 24 jam']"
hos15818801,1,999.0,7.0,Gunung Sindur,"{'Kamar Tidur': 2, 'Kamar Mandi': 2, 'Luas Tanah': '72 m²', 'Luas Bangunan': '53 m²', 'Carport': 2, 'Tipe Properti': 'Rumah', 'Sertifikat': 'SHM - Sertifikat Hak Milik', 'Daya Listrik': '2200 Watt', 'Kamar Pembantu': None, 'Kamar Mandi Pembantu': None, 'Dapur': 1, 'Ruang Makan': 'Ya', 'Ruang Tamu': 'Ya', 'Kondisi Perabotan': 'Unfurnished', 'Material Bangunan': 'Bata Hebel', 'Material Lantai': 'Granit', 'Garasi': 2, 'Jumlah Lantai': 2, 'Hadap': 'Utara', 'Konsep dan Gaya Rumah': 'Minimalis Modern', 'Pemandangan': None, 'Terjangkau Internet': 'Ya', 'Lebar Jalan': '2 Mobil', 'Tahun Dibangun': 2023, 'Sumber Air': 'Sumur Bor', 'Hook': 'Tidak', 'Kondisi Properti': 'Baru', 'ID Iklan': 'hos15818801', 'Tahun Di Renovasi': None}","['Ac', 'Jalur Telepon', 'Jogging Track', 'Keamanan', 'Kolam Renang', 'Taman', 'Tempat Jemuran', 'Wastafel', 'Tempat Gym', 'Track Lari']"


In [9]:
%sql SELECT address_kecamatan, COUNT(*) AS total FROM houses_v1 GROUP BY address_kecamatan ORDER BY total DESC

Running query in 'duckdb'

address_kecamatan,total
Sentul City,1336
Cibinong,669
Cileungsi,583
Kota Wisata,572
Cibubur,362
Bojong Gede,315
Tanah Sareal,311
Gunung Sindur,273
Gunung Putri,266
Sentul,226


In [10]:
%sql SUMMARIZE SELECT * FROM houses_v1

Running query in 'duckdb'

column_name,column_type,min,max,approx_unique,avg,std,q25,q50,q75,count,null_percentage
id,VARCHAR,hos10007095,hos9969539,9012,None,None,None,None,None,8981,0.0%
count(images),BIGINT,1,2,2,1.007348847567086,0.08541460267266754,1,1,1,8981,0.0%
price,DOUBLE,20.0,995000.0,774,45853.76135857461,93606.45982715579,587.4553476397816,4871.297471374046,40269.8937087044,8981,0.01%
installment,DOUBLE,1.0,970.0,174,24.910366328916602,76.8605575420031,3.998546511627907,7.878017998244074,18.625651655046795,8981,0.0%
address_kecamatan,VARCHAR,Ahmadyani,Wanaherang,143,None,None,None,None,None,8981,0.0%
specs,"STRUCT(""Kamar Tidur"" BIGINT, ""Kamar Mandi"" BIGINT, ""Luas Tanah"" VARCHAR, ""Luas Bangunan"" VARCHAR, Carport BIGINT, ""Tipe Properti"" VARCHAR, Sertifikat VARCHAR, ""Daya Listrik"" VARCHAR, ""Kamar Pembantu"" BIGINT, ""Kamar Mandi Pembantu"" BIGINT, Dapur BIGINT, ""Ruang Makan"" VARCHAR, ""Ruang Tamu"" VARCHAR, ""Kondisi Perabotan"" VARCHAR, ""Material Bangunan"" VARCHAR, ""Material Lantai"" VARCHAR, Garasi BIGINT, ""Jumlah Lantai"" BIGINT, Hadap VARCHAR, ""Konsep dan Gaya Rumah"" VARCHAR, Pemandangan VARCHAR, ""Terjangkau Internet"" VARCHAR, ""Lebar Jalan"" VARCHAR, ""Tahun Dibangun"" BIGINT, ""Sumber Air"" VARCHAR, Hook VARCHAR, ""Kondisi Properti"" VARCHAR, ""ID Iklan"" VARCHAR, ""Tahun Di Renovasi"" BIGINT)","{'Kamar Tidur': 1, 'Kamar Mandi': 1, 'Luas Tanah': 102 m², 'Luas Bangunan': 27 m², 'Carport': 1, 'Tipe Properti': Rumah, 'Sertifikat': SHM - Sertifikat Hak Milik, 'Daya Listrik': 1300 Watt, 'Kamar Pembantu': NULL, 'Kamar Mandi Pembantu': NULL, 'Dapur': 1, 'Ruang Makan': Ya, 'Ruang Tamu': Ya, 'Kondisi Perabotan': Unfurnished, 'Material Bangunan': Bata Hebel, 'Material Lantai': Keramik, 'Garasi': NULL, 'Jumlah Lantai': 1, 'Hadap': Utara, 'Konsep dan Gaya Rumah': Minimalis Modern, 'Pemandangan': Perkotaan, 'Terjangkau Internet': Ya, 'Lebar Jalan': 3 Mobil, 'Tahun Dibangun': 2020, 'Sumber Air': PAM atau PDAM, 'Hook': Tidak, 'Kondisi Properti': Bagus, 'ID Iklan': hos14745483, 'Tahun Di Renovasi': 2022}","{'Kamar Tidur': NULL, 'Kamar Mandi': NULL, 'Luas Tanah': 98 m², 'Luas Bangunan': 89 m², 'Carport': NULL, 'Tipe Properti': Rumah, 'Sertifikat': SHM - Sertifikat Hak Milik, 'Daya Listrik': 2200 Watt, 'Kamar Pembantu': NULL, 'Kamar Mandi Pembantu': NULL, 'Dapur': NULL, 'Ruang Makan': NULL, 'Ruang Tamu': Tidak, 'Kondisi Perabotan': NULL, 'Material Bangunan': NULL, 'Material Lantai': NULL, 'Garasi': NULL, 'Jumlah Lantai': 1, 'Hadap': NULL, 'Konsep dan Gaya Rumah': NULL, 'Pemandangan': Pegunungan, 'Terjangkau Internet': Ya, 'Lebar Jalan': 4 Mobil, 'Tahun Dibangun': NULL, 'Sumber Air': PAM atau PDAM, 'Hook': Tidak, 'Kondisi Properti': Baru, 'ID Iklan': hos14191530, 'Tahun Di Renovasi': NULL}",8910,None,None,None,None,None,8981,0.0%
facilities,VARCHAR[],[],"[bisa masuk parkir 12 mobil, Carport, Garden, Swimming Pool]",3532,None,None,None,None,None,8981,0.0%
